In [2]:
from ingestion import extract
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

In [3]:
extract()

,Open,High,Low,Close,Volume,barclays_Open,barclays_High,barclays_Low,barclays_Close,barclays_Volume,...,honda_Open,honda_High,honda_Low,honda_Close,honda_Volume,sony_Open,sony_High,sony_Low,sony_Close,sony_Volume
Gmt time,,,,,,,,,,,,,,,,,,,,,
2013-12-31 22:00:00,174.260,174.260,174.260,174.260,0.0000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-01 22:00:00,174.329,174.834,171.834,172.381,147167.3199,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-02 22:00:00,167.644,167.644,167.644,167.644,0.0000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03 22:00:00,170.353,170.353,170.353,170.353,0.0000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-04 21:00:00,172.352,173.000,172.295,172.705,109723.2202,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03 21:00:00,188.691,189.541,188.407,189.056,142985.6706,177.455,179.254,174.990,177.009,1.068048e+09,...,1750.0,1759.0,1709.0,1732.0,2.762983e+08,12996.0,13088.0,12901.0,13027.0,3.857327e+07
2024-12-04 21:00:00,190.812,190.812,190.812,190.812,0.0000,182.813,182.813,182.813,182.813,0.000000e+00,...,1875.0,1875.0,1875.0,1875.0,0.000000e+00,13014.0,13014.0,13014.0,13014.0,0.000000e+00
2024-12-05 21:00:00,194.753,196.276,194.706,196.153,133106.0806,213.837,216.099,213.343,214.239,6.837120e+08,...,1834.0,1834.0,1748.0,1757.0,3.639180e+08,11815.0,11977.0,11724.0,11920.0,1.858355e+07


In [ ]:
transform_and_scale()

In [ ]:
def prepare_sequences(df: pd.DataFrame, target_column = "Close", sequence_length = 30):
    """
    Prepares sequences from the DataFrame for LSTM input.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame containing the features and target.
    target_column : str
        The name of the target column.
    sequence_length : int
        The length of the sequences.

    Returns
    -------
    tuple
        The sequences for X (features) and y (target).
    """
    feature_columns = df.columns[df.columns != target_column]
    features = df[feature_columns].values
    target = df[target_column].values

    generator = TimeseriesGenerator(features, target, length=sequence_length, batch_size=1)

    X = []
    y = []

    for X_batch, y_batch in generator:
        X.append(X_batch[0])
        y.append(y_batch[0])

    return np.array(X), np.array(y)

In [ ]:
def build_and_compile_model(input_shape):
    """
    Builds and compiles the LSTM model.

    Parameters
    ----------
    input_shape : tuple
        The shape of the input data.

    Returns
    -------
    model : Sequential
        The compiled LSTM model.
    """

    model = Sequential()

    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))

    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(25))

    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
#### this is for the main function #####

# Load and preprocess the data
raw_data = extract()
preprocessed_data = transform(df=raw_data)


# split the data
val_cutoff_date = datetime(2022, 1, 1)  # Define your validation cutoff date
test_cutoff_date = datetime(2023, 1, 1)  # Define your test cutoff date
train_data, val_data, test_data = train_val_test_split(preprocessed_data, val_cutoff_date, test_cutoff_date)

# prepare sequences for LSTM
sequence_length = 60  # Define the length of sequences
X_train, y_train = prepare_sequences(train_data, target_column='Close', sequence_length=sequence_length)
X_val, y_val = prepare_sequences(val_data, target_column='Close', sequence_length=sequence_length)

# build and compile the model
input_shape = (X_train.shape[1], X_train.shape[2])

model = build_and_compile_model(input_shape)


# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',    # Metric to monitor (can be 'val_loss' or 'val_mse', etc.)
    patience=10,            # Number of epochs to wait for improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)


# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val)
)






# Print model summary
model.summary()
model.save('my_lstm_model.h5')
##### load and predict on validation ####

# Load the model from the file
loaded_model = load_model('my_lstm_model.h5')

predictions = loaded_model.predict(X_val)


In [ ]:
##check mse and plot ##
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


# Assuming 'loaded_model' is your model and 'X_val' is your validation input data
# and 'y_val' is your actual validation target data.

# Make predictions
predictions = loaded_model.predict(X_val)

# Calculate MSE
mse = mean_squared_error(y_val, predictions)
print(f"Mean Squared Error: {mse}")

# Plot validation vs predictions
plt.figure(figsize=(14, 7))

# Plot actual values
plt.plot(y_val, label='Actual Values', color='blue')

# Plot predicted values
plt.plot(predictions, label='Predicted Values', color='red', linestyle='--')

plt.title('Validation vs Predictions')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()